<a href="https://colab.research.google.com/github/dhruvkhanna98/Pneumonia-Detector/blob/master/Pneumonia_OG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')


In [0]:
import numpy as np 
import torch
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from collections import OrderedDict
from torch import nn 
from torch import optim
import pandas as pd

In [0]:
# Setting Float Cuda Tensor as Default tensor 
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [0]:
data_dir = 'drive/My Drive/PnemoniaGang/chest_xray'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/val'
test_dir = data_dir + '/test'

In [0]:
train_transforms = transforms.Compose([transforms.Resize((385,385)), 
                                       transforms.RandomRotation(degrees = [-30, 30]),
                                       transforms.ToTensor(), 
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

transforms = transforms.Compose([transforms.Resize((384,384)),
                                 transforms.ToTensor(), 
                                 transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])


# Loading the Data 
train_data = datasets.ImageFolder(train_dir, transform = transforms)

validation_data = datasets.ImageFolder(valid_dir, transform = transforms)

test_data = datasets.ImageFolder(test_dir, transform = transforms)

# DataLoaders 
trainloader = torch.utils.data.DataLoader(train_data, batch_size = 32, shuffle = True, num_workers = 2)

validloader = torch.utils.data.DataLoader(validation_data, batch_size = 32, num_workers = 2)

testloader = torch.utils.data.DataLoader(test_data, batch_size = 32, num_workers = 2)


In [0]:
# Loading a Pre-trained Model
model = models.densenet161(pretrained = True)
model

In [0]:
# Freezing Parameters
for param in model.parameters(): 
    param.requires_grad = False
    
for param in model.features.denseblock4.parameters():
    param.requires_grad = True

input_size = model.classifier.in_features
output_size = 2
hidden_size = 600

classifier = nn.Sequential(nn.Linear(input_size, hidden_size),
                           nn.ReLU(),
                           nn.Dropout(p=0.5),
                           nn.BatchNorm1d(hidden_size),
                           nn.Linear(hidden_size, output_size),
                           nn.LogSoftmax(dim = 1))

model.classifier = classifier

In [0]:
def validation(model, validloader, criterion): 
    
    total = 0
    correct = 0
    validation_loss = 0
    
    for inputs, labels in validloader: 
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        outputs = model(inputs).to("cuda")
        _, predicted = torch.max(outputs.data, 1)
        total += float(labels.size(0))
        correct += float((predicted == labels).sum().item())
        validation_loss += float(criterion(outputs, labels).item())
    return validation_loss, correct, total


In [0]:
# Weights to compensate for the data Imbalance in the Testset classes

input_weight = 3875/5216
label_weight = 1

weights = torch.cuda.FloatTensor([input_weight, label_weight])
weights

In [0]:
criterion = nn.NLLLoss(weight = weights)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 2, gamma=0.1)


In [0]:
steps = 0
test_loss = 0
accuracy = 0
print_every = 40
epochs = 5

# Moving Model to GPU
model.to('cuda')

for e in range(epochs): 
    running_loss = 0
    
    # Decaying Learning Rate
    scheduler.step()
    
    for inputs, labels in trainloader: 
        steps += 1
        
        # Moving inputs and labels to GPU
        inputs, labels = inputs.to('cuda'), labels.to('cuda')
        
        # Forward Pass on the Data
        outputs = model.forward(inputs)
        
        # Loss after Forward Pass
        loss = criterion(outputs, labels)
        
        # Backpropagation
        loss.backward()
        
        # Updating Weights
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0: 
                # Model in Eval Mode for inference
                model.eval()
            
                # Turning off gradients for validation
                with torch.no_grad():
                    validation_loss, correct, total = validation(model, testloader, criterion)
                
                    print("Epoch: {}/{}.. ".format(e+1, epochs),
                          "Training Loss: {:.3f}.. ".format(running_loss/print_every),
                          "Testing Loss: {:.3f}.. ".format(validation_loss/len(testloader)),
                          "Testing Accuracy: {:.3f}".format((correct/total)*100))
                
                    running_loss = 0
                
                # Putting Model back in Training mode
                model.train()

In [0]:
model.eval()
test_loss, correct, total = validation(model, testloader, criterion)
print("Test Loss: {:.3f}.. ".format(test_loss/len(testloader)), "Test Accuracy: {:.3f}".format((correct/total)*100))  